## Dia 5 (Repaso)

## Creación de puntos aleatorios y joins con Geopandas

Antes de nada recordar que un join es una unión entre dos tablas (o shapes) basado en un campo en común, con lo cual nos permite traer elementos de una tabla/shape a otro.


Esto es una combinación de lo visto los días 2 y 3. Abrimos el shape de fincas con geopandas y usamos el método sample_points cerar puntos aleatorios en cada polígono 

¿Podríamos usar el área del polígono para determinar el número de puntos a crear para que fueran más representativos?

Para ver los joins, vamos a seguir la estrategia que vimos en clase partiendo las fincas según la Titularidad Pública o Privada.

Luego cruzamos los shapes de puntos aleatorios con el ndvi (o el dtm, o el raster que queramos). Una vez que tenemos los puntos vamos a ver que tenemos un problema y es que 


Una vez que tenemos las capas de puntos las volvemos a juntar... (es muy ridículo, pero es solo un ejemplo). En realidad en nuestro caso no se trataría de un join como tal (o sí, podríamos hacer dos join, uno para cada tipo de finca), sino tan solo una concatenación (igual que haríamos con Numpy). La cuestión en cuaqluier caso es que no nos guarda ningún campo del shape original con el que podríamos hacer luego el join.  Así que la mejor solución es hacer un Spatial Join, que es un join basado en las geometrías (en nuestro caso los puntos que caen dentro de cada finca).

In [ ]:
import pandas as pd

# Abrimos el shape
fp = "/home/diego/git/AEPython_2024/data/fincas_ETRS89.shp"
data = gpd.read_file(fp)

# Arreglamos los nombres de las columnas (innecesario para el ejercicio)
colnames = {'FINCA':'FINCAS', 'TITULARIDA':'TITULARIDAD'}
data.rename(columns=colnames, inplace=True)

#Subset de dos nuevos gdfs con las fincas privadas y fincas publicas
privada = data[data['TITULARIDAD'] == 'Privada']
publica = data[data['TITULARIDAD'] == 'Pública']

#Creamos los puntos
pts_privadas = privada.sample_points(25)
pts_publicas = public.sample_points(25)

#Guardamos las fincas a shapefile
pts_privadas.to_file('/home/diego/git/AEPython_2024/data/fprivadas_rpoints.shp')
pts_publicas.to_file('/home/diego/git/AEPython_2024/data/fpublicas_rpoints.shp')

# Cargar los puntos generados y los polígonos (fincas)

# ¿Es esto necesario o nos valdría con las variables pts_privadas o pts_publicas y data ya abiertas?
points_gdf = gpd.read_file('/home/diego/git/AEPython_2024/data/fprivadas_rpoints.shp')
polygons_gdf = gpd.read_file('/home/diego/git/AEPython_2024/data/fincas_ETRS89.shp')

# Comprobamos que tienen el mismo CRS puntos y polígonos
print(points_gdf.crs)  # CRS de los puntos
print(polygons_gdf.crs)  # CRS de los polígonos"""

# Hacer el join utilizando el campo "id" (Join by Attributes)
# Así haríamos el Join por atributos (vemos que es básicamente lenguaje SQL para hacer uniones de tablas
publicas_attr = points_gdf.merge(data[['OBJECTID', 'FINCA']], left_on='FID', right_on='OBJECTID')

# Spatial Join (el más chulo)
# Cargar puntos generados y el shapefile de polígonos
points_gdf = gpd.read_file('/home/diego/git/AEPython_2024/data/fprivadas_rpoints.shp') # Hay que hacer los dos shapes de puntos
polygons_gdf = gpd.read_file('/home/diego/git/AEPython_2024/data/fincas_ETRS89.shp')

# Realizar el spatial join
points_with_attributes = gpd.sjoin(points_gdf, polygons_gdf[['OBJECTID', 'FINCA', 'geometry']], how='left', predicate='within')

# Guardar los puntos con los atributos
points_with_attributes.to_file('/home/diego/git/AEPython_2024/data/fprivadas_rpoints_attr_sj.shp')

# Ver el resultado
#print(points_with_attributes.head())

# Cargar los dos shapefiles
gdf1 = gpd.read_file('/home/diego/git/AEPython_2024/data/fpublicas_rpoints_attr_sj.shp')
gdf2 = gpd.read_file('/home/diego/git/AEPython_2024/data/fprivadas_rpoints_attr_sj.shp')

# Concatenar los dos GeoDataFrames
# Aquí no es un join, sino que estamos juntando dos shapes en un único shape, tan solo hay que concatenar los GDFs
combined_gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))

# Verificar que tienen el mismo CRS (si no, reproyectar)
if gdf1.crs != gdf2.crs:
    gdf2 = gdf2.to_crs(gdf1.crs)

# Guardar el resultado en un nuevo shapefile que contenga todos los puntos
combined_gdf.to_file('/home/diego/git/AEPython_2024/data/fmerged_rpoints_attr_sj.shp')

# Ver el resultado
#print(combined_gdf.head())

## Cruce de los puntos aleatorios dentro de los polígonos de las fncas del END con el NDVI

Usamos los NDVI y Puntos ya creados y usamos el script que vimos en la clase para trabajar con los geometrías de tipo Multipoint (de hecho, en un primer momento nos fallo por eso). Una vez que tenemos todos los puntos de cada polígono que se almacenan en la lista point_values, calculamos su promedio y ese es el valor que vamos a guardar en el shapefile

In [ ]:
import rasterio
import geopandas as gpd
from shapely.geometry import Point


## Necesitamos crear dos archivos con puntos aleatorios dentro de cada finca

# Cargar el archivo de puntos (shapefile)
shapefile_path = '/home/diego/git/AEPython_2024/data/fmerged_rpoints_attr_sj.shp'
points_gdf = gpd.read_file(shapefile_path)

# Cargar el archivo ráster
# raster_path = 'path_to_your_raster.tif'
raster = rasterio.open('/home/diego/git/AEPython_2024/data/ndvi_.tif')

# Crear una lista para almacenar los valores de los puntos
values = []

# Iterar sobre cada geometría en el shapefile
for geom in points_gdf.geometry:
    # Lista para almacenar valores de puntos individuales
    point_values = []

    if geom.geom_type == 'Point':
        # Si es un solo punto, extraer las coordenadas
        coords = (geom.x, geom.y)
        # Obtener el índice de la fila/columna correspondiente a las coordenadas del punto
        row, col = raster.index(coords[0], coords[1])
        # Extraer el valor del ráster en esa posición
        point_values.append(raster.read(1)[row, col])

    elif geom.geom_type == 'MultiPoint':
        # Si es MultiPoint, iterar sobre todos los puntos
        for point in geom.geoms:
            coords = (point.x, point.y)
            row, col = raster.index(coords[0], coords[1])
            # Extraer el valor del ráster en esa posición
            point_values.append(raster.read(1)[row, col])

    # Calcular el valor medio si hay puntos válidos, si no asignar None
    if point_values:
        mean_value = sum(point_values) / len(point_values)
        values.append(mean_value)
    else:
        values.append(None)

# Añadir los valores obtenidos al GeoDataFrame
points_gdf['raster_value'] = values

# Mostrar los valores obtenidos
print(points_gdf[['geometry', 'raster_value']])

# Guardamos el gdf a un nuevo shapefile
points_gdf.to_file('/home/diego/git/AEPython_2024/data/fmerged_rpoints_attr_sj_ndvi.shp')

## Sample points Multibanda

In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

# Cargar el archivo de puntos (shapefile)
shapefile_path = '/home/diego/git/AEPython_2024/data/fmerged_rpoints_attr_sj.shp'
points_gdf = gpd.read_file(shapefile_path)

# Cargar el archivo ráster multibanda
raster_path = 'path_to_your_multiband_raster.tif'
raster = mrs #rasterio.open(raster_path)

# Crear listas para almacenar los valores de cada banda
band_values_lists = {f'b{band_index}': [] for band_index in range(1, raster.count + 1)}

# Iterar sobre cada geometría en el shapefile
for geom in points_gdf.geometry:
    # Lista para almacenar los valores de las bandas para cada punto
    band_values = []

    if geom.geom_type == 'Point':
        # Si es un solo punto, extraer las coordenadas
        coords_list = [(geom.x, geom.y)]
    elif geom.geom_type == 'MultiPoint':
        # Si es MultiPoint, extraer las coordenadas de todos los puntos
        coords_list = [(point.x, point.y) for point in geom.geoms]
    else:
        # Si no es Point ni MultiPoint, continuar con el siguiente
        for band_index in range(1, raster.count + 1):
            band_values_lists[f'b{band_index}'].append(None)
        continue

    # Extraer los valores de todas las bandas para cada punto en coords_list
    for coords in coords_list:
        row, col = raster.index(coords[0], coords[1])
        # Extraer los valores de todas las bandas en esa posición
        point_band_values = []
        for band_index in range(1, raster.count + 1):
            band_value = raster.read(band_index)[row, col]
            point_band_values.append(band_value)
        
        # Añadir los valores de las bandas para este punto
        band_values.append(point_band_values)

    # Calcular el valor medio de todas las bandas si hay puntos válidos, si no asignar None
    if band_values:
        mean_values = [np.mean(b) for b in zip(*band_values)]  # Calcular el promedio para cada banda
        for band_index, mean_value in enumerate(mean_values, start=1):
            band_values_lists[f'b{band_index}'].append(int(mean_value))
    else:
        for band_index in range(1, raster.count + 1):
            band_values_lists[f'b{band_index}'].append(None)

# Añadir los valores de cada banda al GeoDataFrame
for band_index in range(1, raster.count + 1):
    points_gdf[f'b{band_index}'] = band_values_lists[f'b{band_index}']

# Mostrar los valores obtenidos
print(points_gdf[['geometry'] + [f'b{band_index}' for band_index in range(1, raster.count + 1)]])


points_gdf.to_file('/home/diego/git/AEPython_2024/data/fmerged_rpoints_attr_sj_multi3.shp')

## Poligonizando rasters

Vamos a coger el raster que guardamos con las 3 clases de NDVI (4 con el NoData) y vamos a pasarlo a polígonos

### Rasterio

In [21]:
%%timeit

import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Abrir el ráster categórico
with rasterio.open('/media/diego/Datos4/EBD/Cursos/AETPython/ndvirec.tif') as src:
    raster = src.read(1)  # Leer la primera banda
    mask = raster != -9999  # Crear una máscara para los valores no nulos

    # Poligonizar el ráster
    results = (
        {'properties': {'class': v}, 'geometry': shape(s)}
        for s, v in shapes(raster, mask=mask, transform=src.transform)
    )

    # Convertir a GeoDataFrame
    geoms = list(results)
    gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)

# Guardar como shapefile
gdf.to_file('/media/diego/Datos4/EBD/Cursos/AETPython/ndvi_pol_.shp')


8.7 s ± 321 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Gdal python bindings

In [20]:
%%timeit 

from osgeo import gdal, ogr
import numpy as np

# Abrir el ráster con GDAL
raster_ds = gdal.Open('/media/diego/Datos4/EBD/Cursos/AETPython/ndvirec.tif')
band = raster_ds.GetRasterBand(1)

# Crear el shapefile de salida
shapefile = '/media/diego/Datos4/EBD/Cursos/AETPython/ndvi_pol_gdal.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
out_ds = driver.CreateDataSource(shapefile)
out_layer = out_ds.CreateLayer('categories', srs=None)

# Añadir un campo para la categoría
field = ogr.FieldDefn('class', ogr.OFTInteger)
out_layer.CreateField(field)

# Poligonizar el ráster
gdal.Polygonize(band, None, out_layer, 0, [], callback=None)

# Cerrar los datasets
band = None
raster_ds = None
out_ds = None

2.1 s ± 279 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Pure GDAL

In [19]:
%%timeit

!gdal_polygonize -nomask -overwrite /media/diego/Datos4/EBD/Cursos/AETPython/ndvirec.tif /media/diego/Datos4/EBD/Cursos/AETPython/ndvi_pol_truegdal.shp

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
2 s ± 201 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Upside down

## Rasterio

In [ ]:
import rasterio
import geopandas as gpd
from rasterio.features import rasterize

# Abrir el shapefile con geopandas
gdf = gpd.read_file('path/to/polygon.shp')

# Definir los parámetros del ráster de salida
transform = rasterio.transform.from_origin(0, 0, 100, 100)  # Origen (izquierda, superior), tamaño de píxel (x, y)
raster_size = (1000, 1000)  # Tamaño del ráster en píxeles (ancho, alto)

# Obtener las geometrías y los valores
shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf['class']))

# Rasterizar
raster = rasterize(
    shapes,
    out_shape=raster_size,
    transform=transform,
    fill=0,  # Valor para las celdas vacías
    dtype='int32'
)

# Guardar el ráster de salida
with rasterio.open(
    'path/to/output_raster.tif',
    'w',
    driver='GTiff',
    height=raster_size[0],
    width=raster_size[1],
    count=1,
    dtype=raster.dtype,
    crs=gdf.crs,
    transform=transform,
) as dst:
    dst.write(raster, 1)


## Gdal

In [ ]:
from osgeo import gdal, ogr

# Parámetros del ráster de salida
output_raster = 'path/to/output_raster.tif'
x_res = 100  # Resolución en el eje X
y_res = 100  # Resolución en el eje Y
raster_size = (1000, 1000)  # Tamaño del ráster en píxeles (ancho, alto)

# Crear el ráster de salida
raster_ds = gdal.GetDriverByName('GTiff').Create(output_raster, raster_size[0], raster_size[1], 1, gdal.GDT_Int32)
raster_ds.SetGeoTransform((0, x_res, 0, 0, 0, -y_res))  # Define la transformación del ráster (origen y tamaño de los píxeles)

# Abrir el shapefile con OGR
vector_ds = ogr.Open('path/to/polygon.shp')
layer = vector_ds.GetLayer()

# Rasterizar la capa vectorial
gdal.RasterizeLayer(raster_ds, [1], layer, options=["ATTRIBUTE=class"])  # 'class' es el nombre del campo en el shapefile que quieres rasterizar

# Cerrar los datasets
raster_ds = None
vector_ds = None
